# コイントスの例

In [ ]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

# ダミーデータの生成 (1: 表, 0: 裏)
# ここでは100回中、60回表が出たとします。
data = np.concatenate([np.ones(60), np.zeros(40)])

with pm.Model() as coin_flip_model:
    # 事前分布: Uniform(0, 1)
    p = pm.Uniform("p", 0, 1)
    
    # 尤度関数: 二項分布
    y = pm.Binomial("y", n=len(data), p=p, observed=data.sum())
    
    # 事後分布のサンプリング
    trace = pm.sample(2000, tune=1000, chains=2)

pm.traceplot(trace)
plt.show()

# 事後分布の平均値と95%信頼区間を表示
pm.summary(trace).round(2)

# コインの山から1枚コインを選びコイントスをする試行を繰り返す例

## データ作成

In [ ]:
import numpy as np

np.random.seed(0)  # 結果を再現可能にするためのシード

# パラメータ設定
n_trials = 10  # 試行回数
n_flips = 100  # 1試行あたりのコイン投げ回数
p_true = 0.5   # コインが表になる真の確率

# ダミーデータ生成
data = np.random.binomial(n=n_flips, p=p_true, size=n_trials)

print("各試行における表が出た回数:")
print(data)

## MCMC

In [ ]:
import pymc3 as pm
import numpy as np

#ダミーデータ生成 (前回のコードに基づいています)
np.random.seed(0)
n_trials = 10
n_flips = 100
p_true = 0.5
data = np.random.binomial(n=n_flips, p=p_true, size=n_trials)

with pm.Model() as hierarchical_model:
    # ハイパーパラメータの事前分布(半コーシー分布)
    alpha = pm.HalfCauchy('alpha', beta=1)
    beta = pm.HalfCauchy('beta', beta=1)
    
    # 各コインの表が出る確率に対する事前分布(ベータ分布)
    theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=n_trials)
    
    # 尤度関数(二項分布)
    y = pm.Binomial('y', n=n_flips, p=theta, observed=data)
    
    # サンプリング
    trace = pm.sample(2000, tune=2000)

#結果の要約とプロット
pm.traceplot(trace)
pm.summary(trace).round(2)


## 変分ベイズ近似

In [ ]:
import pymc3 as pm
import numpy as np
import arviz as az


np.random.seed(0)
n_trials = 10
n_flips = 100
p_true = 0.5
data = np.random.binomial(n=n_flips, p=p_true, size=n_trials)

with pm.Model() as hierarchical_model:
    # ハイパーパラメータの事前分布(半コーシー分布)
    alpha = pm.HalfCauchy('alpha', beta=1)
    beta = pm.HalfCauchy('beta', beta=1)
    
    # 各コインの表が出る確率に対する事前分布(ベータ分布)
    theta = pm.Beta('theta', alpha=alpha, beta=beta, shape=n_trials)
    
    # 尤度関数(二項分布)
    y = pm.Binomial('y', n=n_flips, p=theta, observed=data)
    
    # 変分ベイズで近似
    approx = pm.fit(n=30000, method='advi')

#トレースを抽出し、プロット
trace = approx.sample(draws=5000)
az.plot_trace(trace)
